## Linking with Spark
driver 와 worker들의 python version 이 동일해야한다.

In [1]:
from pyspark import SparkContext, SparkConf

## Initializing Spark 
### SparkContext
어떻게 cluster 에 접근할지 Spark 에게 알려줌
### SparkConf
내 application 정보 저장

In [2]:
conf = SparkConf().setAppName("RDD Programming Guide").setMaster("local")
sc = SparkContext(conf=conf)

25/04/16 00:00:22 WARN Utils: Your hostname, gimjunhaui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.45.202 instead (on interface en0)
25/04/16 00:00:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 00:00:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
